## Library

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from DARNN_model import *
from keras.layers import LSTM


## Data and parameters

In [2]:
# enc_Data = pd.read_csv("../../data/encoder_data.csv")
# dec_Data = pd.read_csv("../../data/decoder_data.csv")
# target = pd.read_csv("../../data/target.csv")

enc_Data = np.load("../../data/encoder_data.npy")
dec_Data = np.load("../../data/decoder_data.npy")
target = np.load("../../data/target.npy")

T = 5
m = 12
p = 12
BATCH_SIZE = 32
split = 48052
LR = 1e-3
EPOCH = 100
hidden_size = 12
sequence_length = 5
num_layers = 2
criterion = nn.MSELoss()
new_model_train = True
model_type = "DARNN(T=T, m=m, p=p)"
save_model_path = f"./model/{model_type}_{T}_{EPOCH}.pt"

In [3]:
train_DS = TensorDataset(torch.from_numpy(enc_Data[:split]), torch.from_numpy(dec_Data[:split]), torch.from_numpy(target[:split]))
test_DS = TensorDataset(torch.from_numpy(enc_Data[:split]), torch.from_numpy(dec_Data[:split]), torch.from_numpy(target[:split]))

train_DL = DataLoader(train_DS, batch_size = 32)
test_DL = DataLoader(test_DS, batch_size = 32)

## Train & Test

In [4]:
# Train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #gpu 활성화 확인
model = DARNN(T=T, m=m, p=p).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)

loss_graph = []
n = len(train_DL)

model.train()
for epoch in range(EPOCH):
    running_loss = 0
    for enc_data, dec_data, labels in train_DL:
        seq, target = [enc_data, dec_data], labels
        
        out = model(seq)
        loss = criterion(out, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    loss_graph.append(running_loss/n)
    if epoch % 20 == 0:
        print("[epoch: %d] loss : %.4f" %(epoch,running_loss/n))

RuntimeError: input.size(-1) must be equal to input_size. Expected 12, got 11

In [ ]:
# # Test
# concatdata = torch.utils.data.ConcatDataset([train_DS, test_DS])
# data_loader = DataLoader(dataset=concatdata, batch_size=32)

# model.eval()
# with torch.no_grad():
#     pred = []
#     for enc_data, dec_data, labels in data_loader:
#         seq, target = [enc_data, dec_data].to(device), labels.to(device)
#         out = model(seq)
#         pred += out.cpu().tolist()

## Print

In [ ]:
# exec(f"model = {model_type}().to(device)")
# print(model)
# x_batch, _ = next(iter(train_DL))
# print(model(x_batch.to(device)).shape)

In [ ]:
# if new_model_train:
#     optimizer = optim.Adam(model.parameters(), lr=LR)
#     loss_history = Train(model, train_DL, criterion, optimizer, EPOCH)

#     torch.save(model, save_model_path)

#     plt.plot(range(1,EPOCH+1),loss_history)
#     plt.xlabel('Epoch')
#     plt.ylabel('loss')
#     plt.title("Train Loss")
#     plt.grid()

In [ ]:
# load_model = torch.load(save_model_path, map_location=device)

## Comparison real_temp with prediction_temp

In [ ]:
#rescaled_actual = scaler1.inverse_transform(df['rail_temp'][sequence_length:].values.reshape(-1,1))

#rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))

#plt.figure(figsize=(6,5))
#plt.plot(rescaled_actual[1583:2016], color='black', alpha=0.8, linewidth=1, label='measured data')
#plt.plot(rescaled_pred[1583:2016], color='red', linewidth=1, alpha=0.8, label='LSTM_10_2_12')
#plt.axhline(55, 0.0, 1.0, color='orange', linestyle='-.', label='First speed restriction line')
#plt.axhline(60, 0.0, 1.0, color='red', linestyle='--', label='Second speed restriction line')
#plt.legend(loc='lower right')
#plt.ylim(0, 1)
#plt.title("Comparison of epochs for LSTM",fontsize=16)
#plt.xticks([0, 36, 72, 108, 144, 180, 216, 252, 288, 324, 360, 396, 432],labels=['00:00','06:00' ,'12:00','18:00','24:00','06:00' ,'12:00','18:00','24:00','06:00' ,'12:00','18:00','24:00'], rotation=45)
#plt.yticks(fontsize=12)
#plt.ylabel("Temperature (℃)")
#plt.xlabel("Day")